In [1]:
# multi-headed CNN model
from numpy import mean
from numpy import std
from numpy import dstack
from pandas import read_csv
from matplotlib import pyplot
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.models import Model
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import Input
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.utils import to_categorical
from keras.layers.merge import concatenate
from keras.utils.vis_utils import plot_model

In [2]:
# load a single file as a numpy array
def load_file(filepath):
    dataframe = read_csv(filepath, header=None, delim_whitespace=True)
    return dataframe.values

# load a list of files and return as a 3d numpy array
def load_group(filenames, prefix=''):
    loaded = list()
    for name in filenames:
        data = load_file(prefix + name)
        loaded.append(data)
    # stack group so that features are the 3rd dimension
    loaded = dstack(loaded)
    return loaded

# load a dataset group, such as train or test
def load_dataset_group(group, prefix = ""):
    filepath = prefix + group + "/Inertial Signals/"
    # load all 9 files as a single array
    filenames = list()
    # total acceleration
    filenames += ["total_acc_x_" + group + ".txt", "total_acc_y_" + group + ".txt",
                  "total_acc_z_" + group + ".txt"]
    # body acceleration
    filenames += ['body_acc_x_' + group + ".txt", "body_acc_y_" + group + ".txt",
                  "body_acc_z_" + group + ".txt"]
    # body gyroscope
    filenames += ["body_gyro_x_" + group + ".txt", "body_gyro_y_" + group + ".txt",
                  "body_gyro_z_" + group + ".txt"]
    # load input data
    X = load_group(filenames, filepath)
    # load class output
    y = load_file(prefix + group + "/y_" + group + ".txt")
    return X, y

# load the dataset, returns train and test X and y elements
def load_dataset(prefix=''):
    # load all train
    trainX, trainy = load_dataset_group("train", prefix + "HARDataset/")
    print("X-train shape", trainX.shape)
    print("Y-train shape", trainy.shape)
    # load all test
    testX, testy = load_dataset_group("test", prefix + "HARDataset/")
    print("X-test shape", testX.shape)
    print("Y-test shape", testy.shape)
    # zero-offset class values
    trainy = trainy - 1
    testy = testy - 1
    # one hot encode y
    trainy = to_categorical(trainy)
    testy = to_categorical(testy)
    print("\nAfter one-hot encoding")
    print("X-train shape", trainX.shape)
    print("Y-train shape", trainy.shape)
    print("X-test shape", testX.shape)
    print("Y-test shape", testy.shape)
    return trainX, trainy, testX, testy

# standardize data
def scale_data(trainX, testX, standardize):
    # remove overlap
    cut = int(trainX.shape[1] / 2)
    longX = trainX[:, -cut:, :]
    # flatten windows
    longX = longX.reshape((longX.shape[0] * longX.shape[1], longX.shape[2]))
    # flatten train and test
    flatTrainX = trainX.reshape((trainX.shape[0] * trainX.shape[1], trainX.shape[2]))
    flatTestX = testX.reshape((testX.shape[0] * testX.shape[1], testX.shape[2]))
    # standardize
    if standardize:
        s = StandardScaler()
        # fit on training data
        s.fit(longX)
        # apply to training and test data
        longX = s.transform(longX)
        flatTrainX = s.transform(flatTrainX)
        flatTestX = s.transform(flatTestX)
    # reshape
    flatTrainX = flatTrainX.reshape((trainX.shape))
    flatTestX = flatTestX.reshape((testX.shape))
    return flatTrainX, flatTestX

# fit and evaluate a model
def evaluate_model(trainX, trainy, testX, testy):
    verbose, epochs, batch_size = 0, 10, 32
    n_timesteps, n_features, n_outputs = trainX.shape[1], trainX.shape[2], trainy.shape[1]
    # head 1
    inputs1 = Input(shape = (n_timesteps,n_features))
    conv1 = Conv1D(filters = 64, kernel_size = 3, activation = "relu")(inputs1)
    drop1 = Dropout(0.5)(conv1)
    pool1 = MaxPooling1D(pool_size = 2)(drop1)
    flat1 = Flatten()(pool1)
    # head 2
    inputs2 = Input(shape = (n_timesteps,n_features))
    conv2 = Conv1D(filters = 64, kernel_size = 5, activation = "relu")(inputs2)
    drop2 = Dropout(0.5)(conv2)
    pool2 = MaxPooling1D(pool_size = 2)(drop2)
    flat2 = Flatten()(pool2)
    # head 3
    inputs3 = Input(shape = (n_timesteps,n_features))
    conv3 = Conv1D(filters = 64, kernel_size = 11, activation = "relu")(inputs3)
    drop3 = Dropout(0.5)(conv3)
    pool3 = MaxPooling1D(pool_size=2)(drop3)
    flat3 = Flatten()(pool3)
    # merge
    merged = concatenate([flat1, flat2, flat3])
    # interpretation
    dense1 = Dense(100, activation = "relu")(merged)
    outputs = Dense(n_outputs, activation = "softmax")(dense1)
    model = Model(inputs = [inputs1, inputs2, inputs3], outputs = outputs)
    # save a plot of the model
    plot_model(model, show_shapes=True, to_file = "multichannel.png")
    model.compile(loss = "categorical_crossentropy", optimizer = "adam", metrics = ["accuracy"])
    # fit network
    model.fit([trainX,trainX,trainX], trainy, epochs = epochs, batch_size = batch_size, verbose = verbose)
    # evaluate model
    _, accuracy = model.evaluate([testX, testX, testX], testy, batch_size = batch_size, verbose = 0)
    return accuracy
 
# summarize scores
def summarize_results(scores):
    m, s = mean(scores), std(scores)
    print("\nAccuracy: %.3f%% (+/-%.3f)" % (m, s))

# run an experiment
def run_experiment(repeats = 10):
    # load data
    trainX, trainy, testX, testy = load_dataset()
    # repeat experiment
    scores = list()
    print()
    for r in range(repeats):
        score = evaluate_model(trainX, trainy, testX, testy)
        score = score * 100.0
        print("Iteration %d: %.3f" % (r+1, score))
        scores.append(score)
    # summarize results
    summarize_results(scores)

In [3]:
# run the experiment
run_experiment()

X-train shape (7352, 128, 9)
Y-train shape (7352, 1)
X-test shape (2947, 128, 9)
Y-test shape (2947, 1)

After one-hot encoding
X-train shape (7352, 128, 9)
Y-train shape (7352, 6)
X-test shape (2947, 128, 9)
Y-test shape (2947, 6)

Iteration 1: 93.519
Iteration 2: 93.621
Iteration 3: 87.716
Iteration 4: 93.960
Iteration 5: 91.822
Iteration 6: 92.195
Iteration 7: 90.668
Iteration 8: 91.890
Iteration 9: 93.010
Iteration 10: 91.449

Accuracy: 91.985% (+/-1.741)
